# Digital System Design with Chisel

### Counter


# Loading The Chisel Library Into a Notebook

In [93]:
val path = System.getProperty("user.dir") + "/../../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/Digital-System-Design-with-Chisel/01-Digital-System-Design/03-Sequential-Logic/../../resource/chisel_deps.sc"

In [94]:
import chisel3._
import chisel3.stage._
import chisel3.util._

import chisel3._

import chisel3.stage._

import chisel3.util._

# ALU

In [87]:
class ALU[A <: Bits with Num[A]](gen: A, functions: (A, A) => A*) extends RawModule {
  val a, b = IO(Input(gen))
  val opcode = IO(Input(UInt(math.pow(2,functions.size).toInt.W)))
  val c = IO(Output(gen))
    
  private def AddSub[T <: UInt](sel: Bool, a: T, b: T){
    val c = Wire(T)
    c := a + Mux(sel, b, ~(b)) + sel
  }
  
  // filter
  private val addCases = functions.zipWithIndex.map { case (f(ai, bi), i) => (i.U, bi) }
    
  // mux
  private val foo = AddSub(selAdd, a, addCase)

  dontTouch(foo)

  c := foo
}

class MyALU extends ALU[UInt](UInt(8.W), _ + _ , _ - _, _ + _ , _ - _)

cmd87.sc:7: not found: value T
    val c = Wire(T)
                 ^cmd87.sc:12: not found: value f
  private val addCases = functions.zipWithIndex.map { case (f(ai, bi), i) => (i.U, bi) }
                                                            ^cmd87.sc:15: not found: value selAdd
  private val foo = AddSub(selAdd, a, addCase)
                           ^cmd87.sc:15: not found: value addCase
  private val foo = AddSub(selAdd, a, addCase)
                                      ^Compilation Failed

: 

In [ ]:
math.pow(2, 3)

# Counter (Chisel3-Util)

In [87]:
class Counter private (r: Range, oldN: Option[Int] = None) extends AffectsChiselPrefix {
  require(r.length > 0, s"Counter range cannot be empty, got: $r")
  require(r.start >= 0 && r.end >= 0, s"Counter range must be positive, got: $r")

  //lazy val
  private lazy val delta = math.abs(r.step)
  private lazy val width = math.max(log2Up(r.last + 1), log2Up(r.head + 1))
  

  /** Number of counts before the counter resets
    *
    * @note Only defined for ranges starting at zero with steps of size 1. Use [[range]] for other
    * use cases.
    */
  def n: Int = oldN match {
    case Some(x) => x
    case None    =>
      // Reasonable for typical ranges
      require(
        r.start == 0 && r.step == 1,
        s"Counter.n only defined on ranges starting at 0 with step == 1, got $r. " +
          "Use underlying range."
      )
      r.last + 1
  }

  /** Creates a counter with the specified number of steps.
    *
    * @param n number of steps before the counter resets
    */
  def this(n: Int) { this(0 until math.max(1, n), Some(n)) }

  /** The current value of the counter. */
  val value = if (r.length > 1) RegInit(r.head.U(width.W)) else WireInit(r.head.U)

  /** The range of the counter values. */
  def range: Range = r

  /** Increments the counter by a step.
    *
    * @note The incremented value is registered and will be visible on the next clock cycle
    * @return whether the counter will wrap on the next clock cycle
    */
  def inc(): Bool = {
    if (r.length > 1) {
      val wrap = value === r.last.U

      if (r.step > 0) {
        // Increasing range
        value := value + delta.U
      } else {
        // Decreasing range
        value := value - delta.U
      }

      // We only need to explicitly wrap counters that don't start at zero, or
      // end on a power of two. Otherwise we just let the counter overflow
      // naturally to avoid wasting an extra mux.
      if (!(r.head == 0 && isPow2(r.last + delta))) {
        when(wrap) { value := r.head.U }
      }

      wrap
    } else {
      true.B
    }
  }

  /** Resets the counter to its initial value */
  def reset(): Unit = {
    value := r.head.U
  }
  
}

cmd87.sc:1: not found: type AffectsChiselPrefix
class Counter private (r: Range, oldN: Option[Int] = None) extends AffectsChiselPrefix {
                                                                   ^Compilation Failed

: 

In [ ]:
log2Up(31)

# Counter (MyCounter)

In [88]:
class MyCounter(max: Int, min: Int = 0, initial: Int = 0) {

  val width : Int = log2Up(max + 1)

  val en    = Wire(Bool())
  val value = RegInit(initial.S(width.W))
    
  val next   = Wire(SInt(width.W))
  val sum    = Wire(SInt((width+1).W))
  val carryIn = Wire(Bool())
  val carryOut = Wire(Bool())
  
  carryOut := sum(width)

  val one = WireDefault(1.S)
  val zero = WireDefault(0.S)
  val maxValue = WireDefault(max.S)
  val minValue = WireDefault(min.S)

  carryIn := false.B
  next := one
  en := false.B
    
  sum := value + Mux(carryIn, ~next, next)
  when (en){
    value := sum
  }

  // Control
  def inc(): Unit =  {
    next := one
    carryIn := false.B
  }
  
  def inc(stride: Int): Unit =  {
    next := stride.S
    carryIn := false.B
  }

  def dec(): Unit =  {
    next := one
    carryIn := true.B
  }
    
  def dec(stride: Int): Unit =  {
    next := stride.S
    carryIn := true.B
  }
    
  def clear(): Unit = {
    next := zero
  }
  
  // Status
  def isMax(): Bool = {
    val out = Wire(Bool())
    out := value === maxValue
    out
  }
    
//  def isAlmostMax(): Bool = {
//    
//  }
    
  def isMin(): Bool = {
    val out = Wire(Bool())
    out := value === minValue
    out
  }
  
//  def isAlmostMin(): Bool = {
//    
//  }
  
  def isZero(): Bool = {
    val out = Wire(Bool())
    out := value === zero
    out
  }
    
//  def isAlmostZero(): Bool = {
//    
//  }
}

defined class MyCounter

# Counter

In [89]:
class helloworld(width: Int) extends Module{
  val io = IO(new Bundle{
    val en = Input(Bool())
    val opcode = Input(UInt(2.W))
    val out = Output(SInt(8.W))
  })
  
  val counter = new MyCounter(7)
  counter.en := io.en
    
  when(counter.isZero){
    counter.inc()
  }

  when(io.opcode === 1.U){
    counter.dec()
  }
    
  when(io.opcode === 2.U){
    counter.dec(2)
  }
    
  when(io.opcode === 3.U){
    counter.inc(2)
  }
  
  io.out := counter.value

}


visualize(() => new helloworld(8))
println(getVerilog(new helloworld(8)))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: helloworld Pages: 1 -->
 
 
 helloworld 
 
 
 cluster_helloworld 
 
 helloworld 
 
<!-- cluster_helloworld_clock -->
 
 cluster_helloworld_clock 
 
 clock 
 
<!-- cluster_helloworld_reset -->
 
 cluster_helloworld_reset 
 
 reset 
 
<!-- mux_1170687282 -->
 
 mux_1170687282 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld_reset->mux_1170687282 -->
 
 cluster_helloworld_reset->mux_1170687282:select 
 
 
 
<!-- cluster_helloworld_io_en -->
 
 cluster_helloworld_io_en 
 
 io_en 
 
<!-- mux_1976198321 -->
 
 mux_1976198321 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld_io_en->mux_1976198321 -->
 
 cluster_helloworld_io_en->mux_1976198321:select 
 
 
 
<!-- cluster_helloworld_io_opcode -->
 
 cluster_helloworld_io_opcode 
 
 io_opcode 
 
<!-- op_eq_1024 -->
 
 op_eq_1024 
 
 
 ● 
 
 eq 
 
 ● 
 
 1 
 
<!-- cluster_helloworld_io_opcode->op_eq_1024 -->
 
 cluster_helloworld_io_opcode->op_eq_1024:in1 
 
 
 
<!-- op_eq_1025 -->
 
 op_eq_1025 
 
 
 ● 
 
 eq 
 
 ● 
 
 3 
 
<!-- cluster_helloworld_io_opcode->op_eq_1025 -->
 
 cluster_helloworld_io_opcode->op_eq_1025:in1 
 
 
 
<!-- op_eq_1027 -->
 
 op_eq_1027 
 
 
 ● 
 
 eq 
 
 ● 
 
 2 
 
<!-- cluster_helloworld_io_opcode->op_eq_1027 -->
 
 cluster_helloworld_io_opcode->op_eq_1027:in1 
 
 
 
<!-- op_eq_1028 -->
 
 op_eq_1028 
 
 
 ● 
 
 eq 
 
 ● 
 
 3 
 
<!-- cluster_helloworld_io_opcode->op_eq_1028 -->
 
 cluster_helloworld_io_opcode->op_eq_1028:in1 
 
 
 
<!-- op_eq_1029 -->
 
 op_eq_1029 
 
 
 ● 
 
 eq 
 
 ● 
 
 2 
 
<!-- cluster_helloworld_io_opcode->op_eq_1029 -->
 
 cluster_helloworld_io_opcode->op_eq_1029:in1 
 
 
 
<!-- cluster_helloworld_io_out -->
 
 cluster_helloworld_io_out 
 
 io_out 
 
<!-- struct_cluster_helloworld_value -->
 
 struct_cluster_helloworld_value 
 
 
 ● 
 
 value 
 
 ● 
 
<!-- op_add_1032 -->
 
 op_add_1032 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- struct_cluster_helloworld_value->op_add_1032 -->
 
 struct_cluster_helloworld_value:out->op_add_1032:in1 
 
 
 
<!-- op_pad_1036 -->
 
 op_pad_1036 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- struct_cluster_helloworld_value->op_pad_1036 -->
 
 struct_cluster_helloworld_value:out->op_pad_1036:in1 
 
 
 
<!-- op_pad_1039 -->
 
 op_pad_1039 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- struct_cluster_helloworld_value->op_pad_1039 -->
 
 struct_cluster_helloworld_value:out->op_pad_1039:in1 
 
 
 
<!-- cluster_helloworld__T -->
 
 cluster_helloworld__T 
 
 _T 
 
<!-- op_or_1026 -->
 
 op_or_1026 
 
 
 ● 
 
 or 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld__T->op_or_1026 -->
 
 cluster_helloworld__T->op_or_1026:in2 
 
 
 
<!-- op_eq_1024->cluster_helloworld__T -->
 
 op_eq_1024:out->cluster_helloworld__T 
 
 
 
<!-- cluster_helloworld_carryIn -->
 
 cluster_helloworld_carryIn 
 
 carryIn 
 
<!-- mux_101757426 -->
 
 mux_101757426 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_helloworld_carryIn->mux_101757426 -->
 
 cluster_helloworld_carryIn->mux_101757426:select 
 
 
 
<!-- mux_2124691396 -->
 
 mux_2124691396 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- mux_2124691396->cluster_helloworld_carryIn -->
 
 mux_2124691396:out->cluster_helloworld_carryIn 
 
 
 
<!-- op_eq_1025->mux_2124691396 -->
 
 op_eq_1025:out->mux_2124691396:select 
 
 
 
<!-- op_or_1026->mux_2124691396 -->
 
 op_or_1026:out->mux_2124691396:in2 
 
 
 
<!-- op_eq_1027->op_or_1026 -->
 
 op_eq_1027:out->op_or_1026:in1 
 
 
 
<!-- cluster_helloworld_next -->
 
 cluster_helloworld_next 
 
 next 
 
<!-- op_not_1030 -->
 
 op_not_1030 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_helloworld_next->op_not_1030 -->
 
 cluster_helloworld_next->op_not_1030:in1 
 
 
 
<!-- cluster_helloworld_next->mux_101757426 -->
 
 cluster_helloworld_next->mux_101757426:in2 
 
 
 
<!-- mux_219434556 -->
 
 mux_219434556 
 
 
 2 
 
 a? 
 
 ● 
 
 ● 
 
<!-- mux_219434

module helloworld(
  input        clock,
  input        reset,
  input        io_en,
  input  [1:0] io_opcode,
  output [7:0] io_out
);
  reg [2:0] value;
  wire  _T = io_opcode == 2'h1;
  wire  _GEN_6 = io_opcode == 2'h2 | _T;
  wire  carryIn = io_opcode == 2'h3 ? 1'h0 : _GEN_6;
  wire [2:0] _GEN_5 = io_opcode == 2'h2 ? $signed(3'sh2) : $signed(3'sh1);
  wire [2:0] next = io_opcode == 2'h3 ? $signed(3'sh2) : $signed(_GEN_5);
  wire [2:0] _sum_T_1 = ~next;
  wire [2:0] _sum_T_2 = carryIn ? $signed(_sum_T_1) : $signed(next);
  wire [2:0] _sum_T_5 = $signed(value) + $signed(_sum_T_2);
  wire [3:0] sum = {{1{_sum_T_5[2]}},_sum_T_5};
  wire [3:0] _GEN_0 = io_en ? $signed(sum) : $signed({{1{value[2]}},value});
  wire [3:0] _GEN_9 = reset ? $signed(4'sh0) : $signed(_GEN_0);
  assign io_out = {{5{value[2]}},value};
  always @(posedge clock) begin
    value <= _GEN_9[2:0];
  end
endmodule


defined class helloworld

# SRLC

In [90]:
import chisel3.experimental.IntParam

// Verilog Black Box
class BlackBoxSRLC(depth: Int) extends BlackBox(Map("DEPTH" -> IntParam(depth))) with HasBlackBoxInline {
  val io = IO(new Bundle{
    val clock = Input(Clock())
    val en = Input(Bool())
    val in = Input(Bool())
    val addr = Input(UInt(log2Down(depth).W))
    val out = Output(Bool())
  })
  setInline("BlackBoxSRLC.v",
    """module BlackBoxSRLC #(
      |  parameter DEPTH = 16
      |)(
      |  input   clock,
      |  output  out,
      |  input   [$clog2(DEPTH)-1:0] addr,
      |  input   in,
      |  input   en
      |);
      |  reg [DEPTH-1:0] ShiftReg;
      |  always@(posedge clock) begin
      |    if(en) begin
      |      ShiftReg <= {ShiftReg[DEPTH-2:0], in};
      |    end
      |  end
      |  assign out = ShiftReg[addr];
      |endmodule
    """.stripMargin)
}

// SRLC
class SRLC(depth: Int) extends Module{
  val io = IO(new Bundle{
    val en = Input(Bool())
    val in = Input(Bool())
    val addr = Input(UInt(log2Down(depth).W))
    val out = Output(Bool())
  })

  //val reg = Reg(UInt(depth.W))
  //val outMux = Wire(Vec(depth, Bool()))
  //  
  //when(io.en){
  //  reg := Cat(reg(depth-2, 0), io.in)
  //}
  //
  //outMux := reg.asBools
  //io.out := outMux(io.addr)
    
  val BlackBoxSRLC = Module(new BlackBoxSRLC(math.pow(2, io.addr.getWidth).toInt))
  BlackBoxSRLC.io.clock := clock
  BlackBoxSRLC.io.en := io.en
  BlackBoxSRLC.io.in := io.in
  BlackBoxSRLC.io.addr := io.addr
  io.out := BlackBoxSRLC.io.out
    
}

object SRLC{
  def apply(in: Bool, addr: UInt, en: Bool) : Bool = {
    val m = Module(new SRLC(math.pow(2, addr.getWidth).toInt))
    m.io.in := in
    m.io.addr := addr
    m.io.en := en
    m.io.out
  }
}

visualize(() => new SRLC(16))
println(getVerilog(new SRLC(16)))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: SRLC Pages: 1 -->
 
 
 SRLC 
 
 
 cluster_SRLC 
 
 SRLC 
 
 
 cluster_SRLC_BlackBoxSRLC 
 
 
 BlackBoxSRLC 
 
 
 
<!-- cluster_SRLC_clock -->
 
 cluster_SRLC_clock 
 
 clock 
 
<!-- cluster_SRLC_BlackBoxSRLC_clock -->
 
 cluster_SRLC_BlackBoxSRLC_clock 
 
 clock 
 
<!-- cluster_SRLC_clock->cluster_SRLC_BlackBoxSRLC_clock -->
 
 cluster_SRLC_clock->cluster_SRLC_BlackBoxSRLC_clock 
 
 
 
<!-- cluster_SRLC_reset -->
 
 cluster_SRLC_reset 
 
 reset 
 
<!-- cluster_SRLC_io_en -->
 
 cluster_SRLC_io_en 
 
 io_en 
 
<!-- cluster_SRLC_BlackBoxSRLC_en -->
 
 cluster_SRLC_BlackBoxSRLC_en 
 
 en 
 
<!-- cluster_SRLC_io_en->cluster_SRLC_BlackBoxSRLC_en -->
 
 cluster_SRLC_io_en->cluster_SRLC_BlackBoxSRLC_en 
 
 
 
<!-- cluster_SRLC_io_in -->
 
 cluster_SRLC_io_in 
 
 io_in 
 
<!-- cluster_SRLC_BlackBoxSRLC_in -->
 
 cluster_SRLC_BlackBoxSRLC_in 
 
 in 
 
<!-- cluster_SRLC_io_in->cluster_SRLC_BlackBoxSRLC_in -->
 
 cluster_SRLC_io_in->cluster_SRLC_BlackBoxSRLC_in 
 
 
 
<!-- cluster_SRLC_io_addr -->
 
 cluster_SRLC_io_addr 
 
 io_addr 
 
<!-- cluster_SRLC_BlackBoxSRLC_addr -->
 
 cluster_SRLC_BlackBoxSRLC_addr 
 
 addr 
 
<!-- cluster_SRLC_io_addr->cluster_SRLC_BlackBoxSRLC_addr -->
 
 cluster_SRLC_io_addr->cluster_SRLC_BlackBoxSRLC_addr 
 
 
 
<!-- cluster_SRLC_io_out -->
 
 cluster_SRLC_io_out 
 
 io_out 
 
<!-- cluster_SRLC_BlackBoxSRLC_out -->
 
 cluster_SRLC_BlackBoxSRLC_out 
 
 out 
 
<!-- cluster_SRLC_BlackBoxSRLC_out->cluster_SRLC_io_out -->
 
 cluster_SRLC_BlackBoxSRLC_out->cluster_SRLC_io_out

module SRLC(
  input        clock,
  input        reset,
  input        io_en,
  input        io_in,
  input  [3:0] io_addr,
  output       io_out
);
  wire  BlackBoxSRLC_clock;
  wire  BlackBoxSRLC_en;
  wire  BlackBoxSRLC_in;
  wire [3:0] BlackBoxSRLC_addr;
  wire  BlackBoxSRLC_out;
  BlackBoxSRLC #(.DEPTH(16)) BlackBoxSRLC (
    .clock(BlackBoxSRLC_clock),
    .en(BlackBoxSRLC_en),
    .in(BlackBoxSRLC_in),
    .addr(BlackBoxSRLC_addr),
    .out(BlackBoxSRLC_out)
  );
  assign io_out = BlackBoxSRLC_out;
  assign BlackBoxSRLC_clock = clock;
  assign BlackBoxSRLC_en = io_en;
  assign BlackBoxSRLC_in = io_in;
  assign BlackBoxSRLC_addr = io_addr;
endmodule


import chisel3.experimental.IntParam

// Verilog Black Box

defined class BlackBoxSRLC
defined class SRLC
defined object SRLC

# ShiftMem

In [91]:
class ShiftMem[T <: chisel3.Data](gen: T, depth: Int) extends Module{
  val io = IO(new Bundle{
    val en = Input(Bool())
    val in = Input(gen)
    val addr = Input(UInt(log2Down(depth).W))
    val out = Output(gen)
  })

  private val width   : Int = gen.getWidth

  private val inData  : UInt = io.in.asUInt
  private val outData : Vec[Bool] = Wire(Vec(width, Bool()))

  for (idx <- 0 until width){
    outData(idx) := SRLC(inData(idx), io.addr, io.en)
  }
  
  io.out := outData.asTypeOf(gen)

}

object ShiftMem{
  def apply[T <: chisel3.Data](in: T, addr: UInt, en: Bool): T = {
    val shiftMem = Module(new ShiftMem(in.cloneType, math.pow(2, addr.getWidth).toInt))
    shiftMem.io.en := en
    shiftMem.io.addr := addr
    shiftMem.io.in := in
    (shiftMem.io.out)
  }
}

class helloworld[T <: chisel3.Data](gen: T, depth: Int) extends Module{
  val io = IO(new Bundle{
    val en = Input(Bool())
    val in = Input(gen)
    val addr = Input(UInt(log2Down(depth).W))
    val out = Output(gen)
  })

  //val shiftMem = Module(new ShiftMem(UInt(8.W), math.pow(2, io.addr.getWidth).toInt))
  //shiftMem.io <> io
  io.out := ShiftMem(io.in, io.addr, io.en)
}

visualize(() => new helloworld(UInt(8.W), 32))
println(getVerilog(new helloworld(UInt(8.W), 32)))
//visualize(() => new ShiftMem(UInt(8.W), 32))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: helloworld Pages: 1 -->
 
 
 helloworld 
 
 
 cluster_helloworld 
 
 helloworld 
 
 
 cluster_helloworld_io_out_shiftMem 
 
 
 io_out_shiftMem 
 
 
 
<!-- cluster_helloworld_clock -->
 
 cluster_helloworld_clock 
 
 clock 
 
<!-- cluster_helloworld_io_out_shiftMem_clock -->
 
 cluster_helloworld_io_out_shiftMem_clock 
 
 clock 
 
<!-- cluster_helloworld_clock->cluster_helloworld_io_out_shiftMem_clock -->
 
 cluster_helloworld_clock->cluster_helloworld_io_out_shiftMem_clock 
 
 
 
<!-- cluster_helloworld_reset -->
 
 cluster_helloworld_reset 
 
 reset 
 
<!-- cluster_helloworld_io_en -->
 
 cluster_helloworld_io_en 
 
 io_en 
 
<!-- cluster_helloworld_io_out_shiftMem_io_en -->
 
 cluster_helloworld_io_out_shiftMem_io_en 
 
 io_en 
 
<!-- cluster_helloworld_io_en->cluster_helloworld_io_out_shiftMem_io_en -->
 
 cluster_helloworld_io_en->cluster_helloworld_io_out_shiftMem_io_en 
 
 
 
<!-- cluster_helloworld_io_in -->
 
 cluster_helloworld_io_in 
 
 io_in 
 
<!-- cluster_helloworld_io_out_shiftMem_io_in -->
 
 cluster_helloworld_io_out_shiftMem_io_in 
 
 io_in 
 
<!-- cluster_helloworld_io_in->cluster_helloworld_io_out_shiftMem_io_in -->
 
 cluster_helloworld_io_in->cluster_helloworld_io_out_shiftMem_io_in 
 
 
 
<!-- cluster_helloworld_io_addr -->
 
 cluster_helloworld_io_addr 
 
 io_addr 
 
<!-- cluster_helloworld_io_out_shiftMem_io_addr -->
 
 cluster_helloworld_io_out_shiftMem_io_addr 
 
 io_addr 
 
<!-- cluster_helloworld_io_addr->cluster_helloworld_io_out_shiftMem_io_addr -->
 
 cluster_helloworld_io_addr->cluster_helloworld_io_out_shiftMem_io_addr 
 
 
 
<!-- cluster_helloworld_io_out -->
 
 cluster_helloworld_io_out 
 
 io_out 
 
<!-- cluster_helloworld_io_out_shiftMem_io_out -->
 
 cluster_helloworld_io_out_shiftMem_io_out 
 
 io_out 
 
<!-- cluster_helloworld_io_out_shiftMem_io_out->cluster_helloworld_io_out -->
 
 cluster_helloworld_io_out_shiftMem_io_out->cluster_helloworld_io_out

module SRLC(
  input        clock,
  input        io_en,
  input        io_in,
  input  [4:0] io_addr,
  output       io_out
);
  wire  BlackBoxSRLC_clock;
  wire  BlackBoxSRLC_en;
  wire  BlackBoxSRLC_in;
  wire [4:0] BlackBoxSRLC_addr;
  wire  BlackBoxSRLC_out;
  BlackBoxSRLC #(.DEPTH(32)) BlackBoxSRLC (
    .clock(BlackBoxSRLC_clock),
    .en(BlackBoxSRLC_en),
    .in(BlackBoxSRLC_in),
    .addr(BlackBoxSRLC_addr),
    .out(BlackBoxSRLC_out)
  );
  assign io_out = BlackBoxSRLC_out;
  assign BlackBoxSRLC_clock = clock;
  assign BlackBoxSRLC_en = io_en;
  assign BlackBoxSRLC_in = io_in;
  assign BlackBoxSRLC_addr = io_addr;
endmodule
module ShiftMem(
  input        clock,
  input        io_en,
  input  [7:0] io_in,
  input  [4:0] io_addr,
  output [7:0] io_out
);
  wire  outData_0_m_clock;
  wire  outData_0_m_io_en;
  wire  outData_0_m_io_in;
  wire [4:0] outData_0_m_io_addr;
  wire  outData_0_m_io_out;
  wire  outData_1_m_clock;
  wire  outData_1_m_io_en;
  wire  outData_1_m_io_in;
 

defined class ShiftMem
defined object ShiftMem
defined class helloworld

# ShiftFIFO

In [92]:
class ShiftFIFO(depth: Int) extends Module{
  val io = IO(new Bundle{
    val enq = Flipped(DecoupledIO(UInt(8.W)))
    val deq = DecoupledIO(UInt(8.W))
  })
  
  //Module Instantiation (Counter)
  val counter = new MyCounter(depth-1) // 0 to depth-1
  //Module Instantiation (ShiftMem)
  io.deq.bits := ShiftMem(io.enq.bits, counter.value.asUInt, io.enq.fire)
  
  //Status(Counter)
  io.enq.ready :=  counter.isMax()
  io.deq.valid := ~counter.isMin()

  //Control
  when(io.enq.fire ^ io.deq.fire){
    counter.en := true.B  
    when(io.enq.fire){
      counter.inc()
    }
    when(io.deq.fire){
      counter.dec()
    }
  } .otherwise{
    counter.en := false.B
  }
}

visualize(() => new ShiftFIFO(32))
println(getVerilog(new ShiftFIFO(32)))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: ShiftFIFO Pages: 1 -->
 
 
 ShiftFIFO 
 
 
 cluster_ShiftFIFO 
 
 ShiftFIFO 
 
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem 
 
 
 io_deq_bits_shiftMem 
 
 
 
<!-- cluster_ShiftFIFO_clock -->
 
 cluster_ShiftFIFO_clock 
 
 clock 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_clock -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_clock 
 
 clock 
 
<!-- cluster_ShiftFIFO_clock->cluster_ShiftFIFO_io_deq_bits_shiftMem_clock -->
 
 cluster_ShiftFIFO_clock->cluster_ShiftFIFO_io_deq_bits_shiftMem_clock 
 
 
 
<!-- cluster_ShiftFIFO_reset -->
 
 cluster_ShiftFIFO_reset 
 
 reset 
 
<!-- mux_526643058 -->
 
 mux_526643058 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_ShiftFIFO_reset->mux_526643058 -->
 
 cluster_ShiftFIFO_reset->mux_526643058:select 
 
 
 
<!-- cluster_ShiftFIFO_io_enq_valid -->
 
 cluster_ShiftFIFO_io_enq_valid 
 
 io_enq_valid 
 
<!-- op_and_1059 -->
 
 op_and_1059 
 
 
 ● 
 
 and 
 
 ● 
 
 ● 
 
<!-- cluster_ShiftFIFO_io_enq_valid->op_and_1059 -->
 
 cluster_ShiftFIFO_io_enq_valid->op_and_1059:in2 
 
 
 
<!-- op_and_1078 -->
 
 op_and_1078 
 
 
 ● 
 
 and 
 
 ● 
 
 ● 
 
<!-- cluster_ShiftFIFO_io_enq_valid->op_and_1078 -->
 
 cluster_ShiftFIFO_io_enq_valid->op_and_1078:in2 
 
 
 
<!-- cluster_ShiftFIFO_io_enq_bits -->
 
 cluster_ShiftFIFO_io_enq_bits 
 
 io_enq_bits 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_io_in -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_io_in 
 
 io_in 
 
<!-- cluster_ShiftFIFO_io_enq_bits->cluster_ShiftFIFO_io_deq_bits_shiftMem_io_in -->
 
 cluster_ShiftFIFO_io_enq_bits->cluster_ShiftFIFO_io_deq_bits_shiftMem_io_in 
 
 
 
<!-- cluster_ShiftFIFO_io_deq_ready -->
 
 cluster_ShiftFIFO_io_deq_ready 
 
 io_deq_ready 
 
<!-- op_and_1061 -->
 
 op_and_1061 
 
 
 ● 
 
 and 
 
 ● 
 
 ● 
 
<!-- cluster_ShiftFIFO_io_deq_ready->op_and_1061 -->
 
 cluster_ShiftFIFO_io_deq_ready->op_and_1061:in1 
 
 
 
<!-- op_and_1063 -->
 
 op_and_1063 
 
 
 ● 
 
 and 
 
 ● 
 
 ● 
 
<!-- cluster_ShiftFIFO_io_deq_ready->op_and_1063 -->
 
 cluster_ShiftFIFO_io_deq_ready->op_and_1063:in1 
 
 
 
<!-- cluster_ShiftFIFO_io_enq_ready -->
 
 cluster_ShiftFIFO_io_enq_ready 
 
 io_enq_ready 
 
<!-- cluster_ShiftFIFO_io_enq_ready->op_and_1059 -->
 
 cluster_ShiftFIFO_io_enq_ready->op_and_1059:in1 
 
 
 
<!-- cluster_ShiftFIFO_io_enq_ready->op_and_1078 -->
 
 cluster_ShiftFIFO_io_enq_ready->op_and_1078:in1 
 
 
 
<!-- cluster_ShiftFIFO_io_deq_valid -->
 
 cluster_ShiftFIFO_io_deq_valid 
 
 io_deq_valid 
 
<!-- cluster_ShiftFIFO_io_deq_valid->op_and_1061 -->
 
 cluster_ShiftFIFO_io_deq_valid->op_and_1061:in2 
 
 
 
<!-- cluster_ShiftFIFO_io_deq_valid->op_and_1063 -->
 
 cluster_ShiftFIFO_io_deq_valid->op_and_1063:in2 
 
 
 
<!-- cluster_ShiftFIFO_io_deq_bits -->
 
 cluster_ShiftFIFO_io_deq_bits 
 
 io_deq_bits 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_io_en -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_io_en 
 
 io_en 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_io_addr -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_io_addr 
 
 io_addr 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_io_out -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_io_out 
 
 io_out 
 
<!-- cluster_ShiftFIFO_io_deq_bits_shiftMem_io_out->cluster_ShiftFIFO_io_deq_bits -->
 
 cluster_ShiftFIFO_io_deq_bits_shiftMem_io_out->cluster_ShiftFIFO_io_deq_bits 
 
 
 
<!-- struct_cluster_ShiftFIFO_value -->
 
 struct_cluster_ShiftFIFO_value 
 
 
 ● 
 
 value 
 
 ● 
 
<!-- op_add_1067 -->
 
 op_add_1067 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- struct_cluster_ShiftFIFO_value->op_add_1067 -->
 
 struct_cluster_ShiftFIFO_value:out->op_add_1067:in1 
 
 
 
<!-- op_eq_1071 -->
 
 op_eq_1071 
 
 
 ● 
 
 eq 
 
 ● 
 
 0 
 
<!-- struct_cluster_ShiftFIFO_value->op_eq_1071 -->
 
 struct_cluster_ShiftFIFO_value:out->op_eq_1071:in1 

module SRLC(
  input        clock,
  input        io_en,
  input        io_in,
  input  [4:0] io_addr,
  output       io_out
);
  wire  BlackBoxSRLC_clock;
  wire  BlackBoxSRLC_en;
  wire  BlackBoxSRLC_in;
  wire [4:0] BlackBoxSRLC_addr;
  wire  BlackBoxSRLC_out;
  BlackBoxSRLC #(.DEPTH(32)) BlackBoxSRLC (
    .clock(BlackBoxSRLC_clock),
    .en(BlackBoxSRLC_en),
    .in(BlackBoxSRLC_in),
    .addr(BlackBoxSRLC_addr),
    .out(BlackBoxSRLC_out)
  );
  assign io_out = BlackBoxSRLC_out;
  assign BlackBoxSRLC_clock = clock;
  assign BlackBoxSRLC_en = io_en;
  assign BlackBoxSRLC_in = io_in;
  assign BlackBoxSRLC_addr = io_addr;
endmodule
module ShiftMem(
  input        clock,
  input        io_en,
  input  [7:0] io_in,
  input  [4:0] io_addr,
  output [7:0] io_out
);
  wire  outData_0_m_clock;
  wire  outData_0_m_io_en;
  wire  outData_0_m_io_in;
  wire [4:0] outData_0_m_io_addr;
  wire  outData_0_m_io_out;
  wire  outData_1_m_clock;
  wire  outData_1_m_io_en;
  wire  outData_1_m_io_in;
 

defined class ShiftFIFO